# Análisis exploratorio

## Carga de los datos

In [2]:
import pandas as pd
from pathlib import Path

In [3]:
directorio = Path('../datos')
dataframes = {}
for f in directorio.iterdir():
    print(f)
    if f.suffix == '.xlsx':
        xlsx = pd.ExcelFile(f)
        for sheet in xlsx.sheet_names:
            if sheet != 'WData':
                df = xlsx.parse(sheet)
                dataframes[str(f)+"/"+sheet] = df

../datos/Centrífuga 17825.xlsx
../datos/Biorreactor 13169.xlsx
../datos/Cinéticos IPC.xlsx
../datos/Horas inicio fin centrífugas.xlsx
../datos/Fases producción.xlsx
../datos/Centrífuga 14246.xlsx
../datos/Biorreactor 14617.xlsx
../datos/Biorreactor 13171.xlsx
../datos/Biorreactor 14614.xlsx
../datos/Biorreactor 13170.xlsx
../datos/Biorreactor 14615.xlsx
../datos/Biorreactor 14618.xlsx
../datos/Movimientos componentes.xlsx
../datos/Biorreactor 13172.xlsx
../datos/Temperaturas y humedades.xlsx
../datos/URL descripcion reto.txt
../datos/Biorreactor 14616.xlsx
../datos/OF 123456.xlsx
../datos/Fases producción_test.xlsx
../datos/Centrífuga 12912.xlsx


In [5]:
dataframes.keys()

dict_keys(['../datos/Centrífuga 17825.xlsx/Datos', '../datos/Biorreactor 13169.xlsx/Datos', '../datos/Cinéticos IPC.xlsx/Inóculos', '../datos/Cinéticos IPC.xlsx/Cultivos finales', '../datos/Cinéticos IPC.xlsx/Centrifugación', '../datos/Horas inicio fin centrífugas.xlsx/Hoja1', '../datos/Fases producción.xlsx/Preinóculo', '../datos/Fases producción.xlsx/Inóculo', '../datos/Fases producción.xlsx/Cultivo final', '../datos/Centrífuga 14246.xlsx/Datos', '../datos/Biorreactor 14617.xlsx/Datos', '../datos/Biorreactor 13171.xlsx/Datos', '../datos/Biorreactor 14614.xlsx/Datos', '../datos/Biorreactor 13170.xlsx/Datos', '../datos/Biorreactor 14615.xlsx/Datos', '../datos/Biorreactor 14618.xlsx/Datos', '../datos/Movimientos componentes.xlsx/Full1', '../datos/Biorreactor 13172.xlsx/Datos', '../datos/Temperaturas y humedades.xlsx/Datos', '../datos/Biorreactor 14616.xlsx/Datos', '../datos/OF 123456.xlsx/Sheet1', '../datos/Fases producción_test.xlsx/Cultivo final', '../datos/Centrífuga 12912.xlsx/Datos

In [15]:
dataframes['../datos/Fases producción.xlsx/Inóculo']

,LOTE,ID bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de cultivo,Turbidez inicio cultivo,Turbidez final culttivo,Viabilidad final cultivo
0,23019,13171,NaT,NaT,NaN,NaN,32.80,134400000
1,23020,13171,NaT,NaT,NaN,NaN,32.80,134400000
2,23021,14618,NaT,NaT,NaN,NaN,27.84,115200000
3,23022,14618,NaT,NaT,NaN,NaN,27.84,115200000
4,23023,14618,2023-03-27 07:22:00,2023-03-28 07:29:00,155.20,18.24,31.68,106400000
...,...,...,...,...,...,...,...,...
163,24101,13171,2024-06-28 07:16:00,2024-06-29 07:06:00,167.52,16.16,28.08,91200000
164,24103,13171,2024-06-28 07:16:00,2024-06-29 07:06:00,167.52,16.16,28.08,91200000
165,24104,13172,2024-07-01 07:01:00,2024-07-02 08:01:00,169.04,14.40,26.32,82400000
166,24105,13172,2024-07-01 07:01:00,2024-07-02 08:01:00,169.04,14.40,26.32,82400000
